In [1]:
!pip install -q langchain-google-genai langchain-community langchain-core requests duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [2]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
import requests

In [4]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

In [14]:
@tool
def getweather(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key=<your-api-key>&query={city}'

  response = requests.get(url)

  return response.json()

In [15]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")

In [16]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [17]:
prompt = hub.pull("hwchase17/react")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [18]:
agent = create_react_agent(
    llm=llm,
    tools=[search_tool,getweather],
    prompt=prompt
)

In [19]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool,getweather],
    verbose=True
)

In [20]:
respons = agent_executor.invoke({"input":"Find the capital of Madhya Pradesh, then find it's current weather condition"})



> Entering new AgentExecutor chain...
I need to first find the capital of Madhya Pradesh using a search engine. Then, I will use the getweather tool to find its current weather.
Action: duckduckgo_search
Action Input: capital of Madhya PradeshIndore (/ ɪ n ˈ d ɔːr / ⓘ; ISO: Indaura, Hindi: [ɪn̪d̪ɔːr]) is the largest and most populous city in the Indian state of Madhya Pradesh.The commercial capital of the state, it has been declared as the cleanest city of India 7 times in a row. [15] It is also considered the largest education hub in central India and houses campuses of both the Indian Institute of Technology and the ... Madhya Pradesh, state of India that is situated in the heart of the country. It has no coastline and no international frontier. Its physiography is characterized by low hills, extensive plateaus, and river valleys. The capital is Bhopal, in the west-central part of the state. The name Indore is due to this diety. It is the largest city in Madhya Pradesh state in cen

In [22]:
respons['output']

'The capital of Madhya Pradesh is Bhopal. The current weather in Bhopal is partly cloudy with a temperature of 33 degrees Celsius.'